In [10]:
import os
path = "/home/parker/supreme_court_transcripts/oyez/cases/"
cases = os.listdir(path)

In [11]:
print(len(cases))

16425


In [12]:
import json

validCases = []

for case in cases:
    try:
        caseDetails = json.load(open(path + case))
        if type(caseDetails) == dict:
            if (caseDetails["facts_of_the_case"] is not None and caseDetails["facts_of_the_case"] != '') and (caseDetails["question"] is not None and caseDetails["question"] != '') and (caseDetails["decisions"] is not None and caseDetails["decisions"] != ''):
                validCases.append(caseDetails)
    except:
        pass

In [13]:
len(validCases)

3122

In [14]:
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  cleantext = cleantext.strip('\n')
  return cleantext

In [15]:
def parseCase(case):
    try:
        cn = case["name"].split("v.")
        plantiff = cn[0].strip()
        defendant = cn[1].strip()
        question = cleanhtml(case["question"])
        facts = cleanhtml(case["facts_of_the_case"])

        question = question.replace(plantiff, "plantiff").replace(defendant, "defendant")
        facts = facts.replace(plantiff, "plantiff").replace(defendant, "defendant")

        winningParty = case["decisions"][0]["winning_party"]
        inFavorPlantiff = True if winningParty == plantiff or winningParty == case["first_party"] else False
        court = case["heard_by"][0]["identifier"]
        votes = {}
        for vote in case["decisions"][0]["votes"]:
            if vote["vote"] != "none":
                if inFavorPlantiff and vote["vote"] == "majority":
                    votes[vote["member"]["identifier"]] = "plantiff"
                elif not inFavorPlantiff and vote["vote"] == "majority":
                    votes[vote["member"]["identifier"]] = "defendant"
                elif inFavorPlantiff and vote["vote"] == "minority":
                    votes[vote["member"]["identifier"]] = "defendant"
                elif not inFavorPlantiff and vote["vote"] == "minority":
                    votes[vote["member"]["identifier"]] = "plantiff"
        return {
            "plantiff": plantiff,
            "defendant": defendant,
            "question": question,
            "facts": facts,
            "court": court,
            "winningParty": winningParty,
            "inFavorPlantiff": inFavorPlantiff,
            "votes": votes
        }
    except:
        pass

In [16]:
processedCases = []
for case in validCases:
    paresed = parseCase(case)
    if paresed is not None:
        processedCases.append(paresed)

In [17]:
len(processedCases)

2938

In [18]:
processedCases[0]

{'plantiff': 'Proffitt',
 'defendant': 'Florida',
 'question': 'Is the death penalty a "cruel and unusual" punishment? Is defendant\'s capital-sentencing procedure unconstitutional?',
 'facts': 'Following his defendant conviction for first-degree murder and the imposition of the death penalty, plantiff challenged the constitutionality of both his death sentence, alleging it was a "cruel and unusual" punishment, and defendant\'s capital-sentencing procedure, alleging is was arbitrary and capricious insofar as it permitted judges rather than juries to act as sole sentencing authorities.\nThis case is one of the five "Death Penalty Cases" along with  Gregg v. Georgia ,  Jurek v. Texas ,  Roberts v. Louisiana , and  Woodson v. North Carolina .',
 'court': 'burger6',
 'winningParty': None,
 'inFavorPlantiff': False,
 'votes': {'potter_stewart': 'defendant',
  'thurgood_marshall': 'plantiff',
  'william_j_brennan_jr': 'plantiff',
  'byron_r_white': 'defendant',
  'warren_e_burger': 'defendan

In [19]:
processedCases[3]

{'plantiff': 'Hosanna-Tabor Evangelical Lutheran Church and School',
 'defendant': 'EEOC',
 'question': 'Does the ministerial exception, which prohibits most employment-related lawsuits against religious organizations by employees performing religious functions, apply to a teacher at a religious elementary school who teaches the full secular curriculum, but also teaches daily religion classes, is a commissioned minister, and regularly leads students in prayer and worship?',
 'facts': 'Cheryl Perich filed a lawsuit against the plantiff in Redford, Mich., for allegedly violating the Americans with Disabilities Act when they fired her after she became sick in 2004. After several months on disability, Perich was diagnosed and treated for narcolepsy and was able to return to work without restrictions. But she said the school at that point urged her to resign and, when she refused, fired her.\nPerich filed a complaint with the Equal Employment Opportunity Commission, which ruled in her favor

In [20]:
import pickle

with open('processedCases.pickle', 'wb') as handle:
    pickle.dump(processedCases, handle, protocol=pickle.HIGHEST_PROTOCOL)